In [ ]:
library(stringr)
library(dplyr)
library(ggplot2)
library(lubridate)
library(ISOweek)
library(scales)
library(zoo)
library(tidyr)
library(quantreg)
library(splines)
library(pbs)
library(forecast)
require(RcppRoll)
library(fpp2)
library(xts)
theme_set(theme_bw())
get_percentage = function(x) {
    (exp(x) - 1) * 100
}

In [ ]:
# You can also redownload the data with the get_data.r script
# The data seems to be
df = read.csv("../data/stmf.csv", skip=1)
n_rows = nrow(df)

In [ ]:
df["Weekday"] = "1"
df$Week = str_pad(df$Week, 2, pad="0")
data = df %>% unite(Weekdate, "Year", "Week", sep="-W", remove=FALSE)
data = data %>% unite(Weekdate, "Weekdate", "Weekday", sep="-")
data$Date = ISOweek2date(data$Weekdate)
death_cols = c("D0_14", "D15_64", "D65_74", "D75_84", "D85p")
rate_cols = c("R0_14", "R15_64", "R65_74", "R75_84", "R85p")
base_cols = c("CountryCode", "Weekdate", "Year", "Week", "Sex")
data = data %>% pivot_longer(all_of(c(death_cols, rate_cols)), names_pattern="(.)(.*)", names_to=c("Type", "Age")) %>% pivot_wider(names_from=Type, values_from=value)# data$Pop = data$Deaths / data$Rates
data = rename(data, Deaths = D)
data = rename(data, Rates = R)
data$Pop = data$Deaths / data$Rates

data = data %>% group_by(Age, Sex, CountryCode) %>% fill(Pop, .direction="up")
data$Pop_int = data$Pop
data$Pop_int[data$Week != 52] = NA
data = data[data$Week != 53, ]
data = data %>%
    group_by(Sex, Age, CountryCode) %>%
    arrange(Date) %>%
    mutate(time=seq(1, n())) %>% 
    mutate(Pop_int=approx(time,Pop_int,time)$y) %>%
    fill(Pop_int, .direction="downup") %>%
    select(-time)
data$Rate_norm = data$Deaths / data$Pop_int

# European Standard population numbers
# Get the ESP groups
std_esp_pop = c(1000, 4000, 5500, 5500, 5500, 6000, 6000, 6500, 7000, 
            7000, 7000, 7000, 6500, 6000, 5500, 5000, 4000, 2500, 1500, 800, 200)
esp_group = cut(c(0:100), c(0,1,seq(5,95, 5), 200), right=FALSE)
age = c("0_14", "15_64", "65_74", "75_84", "85p")
esp_pop = c(sum(std_esp_pop[1:4]), sum(std_esp_pop[5:14]), sum(std_esp_pop[15:16]), sum(std_esp_pop[17:18]), sum(std_esp_pop[19:21]))
esp_df = data.frame(Age=age, Esp_pop=esp_pop)
esp_groups = levels(esp_group)
std_esp_df = data.frame(age=esp_groups, esp_pop=std_esp_pop)
data = merge(data, esp_df, by="Age")
data$Deaths_norm = data$Rate_norm * data$Esp_pop
sprintf("Fraction of data missing %.2f", dim(data[data$Deaths_norm==0,])[1] / dim(data)[1])
# With a smoother we are down to 3%. This is managable
# Define functions
weighted_mean = function(x) {
    nas = is.na(x)
    w = c(0.25, 0.5, 0.25)
    if (all(nas)) {
        return(0)
    } else {
        w = w[!nas] / sum(w[!nas])
        return(sum(x[!nas] * w))
    }
}
sprintf("Smoothing...")
data <- data %>% group_by(Age, CountryCode, Sex) %>% arrange(Sex, Age, Date) %>%
       mutate(Sdeaths_norm=rollapply(Deaths_norm, 3, weighted_mean, align='center', fill=NA))
sprintf("Fraction of data missing after smoothing %.2f", dim(data[data$Sdeaths_norm==0,])[1] / dim(data)[1])

data = data %>% fill(Sdeaths_norm, .direction="downup") %>% filter(Sdeaths_norm > 0)

head(data)

In [ ]:
mortality_fit = function(
    data, 
    # Parameters
    years_per_df = 5,
    df.weekly = 4,
    df.res = 2,
    alpha=0.10,
    response.col = "Sdeaths_norm",
    debug=FALSE,
    cv=FALSE) {
    # This function fits trends and confidence intervals at the alpha level
    # Enforced column names in input data
    
    test_start_year = 2020
    start_year = 2012
    
    week.col = "Week"
    year.col = "Year"
    date.col = "Date"
    sex.col = "Sex"
    country.col = "CountryCode"
    age.col = "Age"

    all = data.frame(date = data[,date.col],
                    week = as.numeric(unlist(data[,week.col])),
                    year = as.numeric(unlist(data[,year.col])),
                    response=log(data[, response.col]),
                    sex = as.factor(unlist(data[,sex.col])),
                    country = as.factor(unlist(data[,country.col])),
                    age = as.factor(unlist(data[,age.col])))
    
    colnames(all) = c("date", "week", "year", "response", "sex", "country", "age")
    df.yearly = floor(max((test_start_year - min(all$year)) / years_per_df, 2))
    print(sprintf("Using %d degrees of freedom in yearly splines for %s, %s, %s", 
            df.yearly, all$country[1], all$age[1], all$sex[1]))
    all = subset(all, year(date) > start_year)
    train = subset(all, year(date) < test_start_year)
    test = subset(all, year(date) >= test_start_year)

    yearly.fit = rq(response ~ ns(date, df=df.yearly), data=train)

    train[, "res.yearly"] = yearly.fit$res
    train[, "fit.yearly"] = yearly.fit$fit
    weekly.fit = rq(res.yearly ~ 0 + pbs(week, df=df.weekly), data=train)
    
    train[, "fit.weekly"] = weekly.fit$fit
    train[, "res.weekly"] = weekly.fit$res
    train = train %>% mutate(centered = response - fit.weekly - fit.yearly)
    # Set test date to last train date to fix yearly trend in test period
    mod_date = all$date
    mod_date[year(mod_date) >= test_start_year] = as.Date("2019-12-31", "%Y-%m-%d")
    mod_all = data.frame(date=mod_date, week=all$week)
    all$fit.yearly = predict(yearly.fit, newdata=mod_all)
    all$fit.weekly = predict(weekly.fit, newdata=mod_all)
    
    all = all %>% mutate(centered = response - fit.weekly - fit.yearly)
    n_train = nrow(train)
    n_all = nrow(all)
    # Simple constant quantile c.i.
    if (df.res <= 2) {
        fit.low = quantile(train$centered, alpha/2)
        fit.high = quantile(train$centered, 1 - alpha/2)
        train$centered_high = rep(fit.high, n_train)
        train$centered_low = rep(fit.low, n_train)
        all$centered_high = rep(fit.high, n_all)
        all$centered_low = rep(fit.low, n_all)
    } else {
        fit.low = rq(centered ~ 0 + pbs(week, df=df.res), data=train, tau=alpha/2)
        fit.high = rq(centered ~ 0 + pbs(week, df=df.res), data=train, tau=1-alpha/2)
        train$centered_high = fit.high$fit
        train$centered_low = fit.low$fit
        all$centered_high = predict(fit.high, newdata=all)
        all$centered_low = predict(fit.low, newdata=all)
    }
    
    test = subset(all, year(date) > 2020)
    frac_outside = sum(train$centered < train$centered_low | train$centered > train$centered_high) / nrow(train)
    frac_outside_test = sum(test$centered < test$centered_low | test$centered > test$centered_high) / nrow(test)
    frac_over_test = sum(test$centered > test$centered_high) / nrow(test)
    if (debug) {
        print(sprintf("Fraction of observations outside confidence band in pre-2020: %.3f", frac_outside))
        print(sprintf("Fraction of observations outside confidence band in 2021: %.3f", frac_outside_test))
        print(sprintf("Fraction of observations over confidence band in 2021: %.3f", frac_over_test))
    }
    if (cv) {
        return(list(df=all, weekly.fit=weekly.fit))
    }
    return(all)
}

plot_debug = function(df) {
    ggplot(df, aes(date, response)) + geom_line(alpha=0.3) + 
    geom_line(aes(date, fit.yearly), size=1.5, color="blue") + 
    geom_line(aes(date, fit.weekly + fit.yearly), size=1.5, color="gold") +
    geom_ribbon(aes(ymin=centered_low + fit.weekly + fit.yearly, 
                    ymax=centered_high + fit.weekly + fit.yearly), alpha=0.3) + 
    labs(x="Date", y="Deaths [per 100k per week]", title="Anomalous Mortality 2012-2022") 
}
plot_mortality = function(df) {
    ggplot(df, aes(date, get_percentage(centered))) + geom_line(alpha=0.3) + 
    geom_smooth(method="lm", formula=y ~ ns(x, df=16), se=FALSE) +
    geom_ribbon(aes(ymin=get_percentage(centered_low), ymax=get_percentage(centered_high)), alpha=0.3) + 
    labs(x="Date", y="Deaths [% from baseline 2014-2019]", title="Anomalous Mortality 2012-2022") 

}

In [ ]:
stratified = data %>% group_by(CountryCode, Sex, Age) %>% group_map(~ mortality_fit(.x, response.col="Sdeaths_norm"), keep=TRUE)
stratified = do.call(rbind, stratified)
# stratified_dfres = data %>% group_by(CountryCode, Sex, Age) %>% group_map(~ mortality_fit(.x, response.col="Sdeaths_norm", df.res=6), keep=TRUE)
# stratified_dfres = do.call(rbind, stratified_dfres)

In [ ]:
# Subset
options(repr.plot.width=9, repr.plot.height=4)
plot_mortality(subset(stratified, country=="USA" & age=="15_64" & sex=="b" & year >= 2016))
# plot_mortality(subset(stratified_dfres, country=="USA" & age=="15_64" & sex=="b" & year >= 2016))
# plot_mortality(subset(stratified, country==country & age=="65_74" & sex=="b"))

In [ ]:
# Subset
options(repr.plot.width=9, repr.plot.height=4)
# cdata = subset(stratified, country=="NOR" & age=="65_74" & sex=="b" & Sdeaths_norm > 0)
# all = mortality_fit(cdata, response.col="Sdeaths_norm")
countrysub = subset(stratified, country=="USA" & age=="65_74" & sex=="b" & year >= 2012)
plot_mortality(countrysub)
plot_debug(countrysub)

In [ ]:
# Sum age groups' deaths and population
high_ages = subset(data, Age %in% c("65_74", "74_85", "85p")) %>% group_by(Date, CountryCode, Sex) %>% 
    summarise(Sdeaths_norm=sum(Sdeaths_norm),
             Date=Date, Week=Week, Year=Year, Age="all") %>% distinct()
mid_ages = subset(data, Age %in% c("15_64")) %>% group_by(Date, CountryCode, Sex) %>% 
    summarise(Sdeaths_norm=sum(Sdeaths_norm),
             Date=Date, Week=Week, Year=Year, Age="all") %>% distinct()
adult_ages = subset(data, Age %in% c("15_64","65_74", "74_85", "85p")) %>% group_by(Date, CountryCode, Sex) %>% 
    summarise(Sdeaths_norm=sum(Sdeaths_norm),
             Date=Date, Week=Week, Year=Year, Age="all") %>% distinct()

In [ ]:
# Fit all countries
res_high = high_ages %>% group_by(CountryCode, Sex) %>% group_map(~ mortality_fit(.x), keep=TRUE)
high_ages_df = do.call(rbind, res_high)
res_mid = mid_ages %>% group_by(CountryCode, Sex) %>% group_map(~ mortality_fit(.x), keep=TRUE)
mid_ages_df = do.call(rbind, res_mid)
res_adult = adult_ages %>% group_by(CountryCode, Sex) %>% group_map(~ mortality_fit(.x), keep=TRUE)
adult_ages_df = do.call(rbind, res_adult)

all_ages_df = rbind(high=high_ages_df, mid=mid_ages_df)
all_ages_df = rbind(high=high_ages_df, mid=mid_ages_df, adult=adult_ages_df)
all_ages_df$age_group = rownames(all_ages_df)
all_ages_df$age_group = gsub("\\.*\\d", "", all_ages_df$age_group)
all_ages_df$age_group = gsub(".Sex", "", all_ages_df$age_group)

In [ ]:
options(repr.plot.width=25, repr.plot.height=10)
p = ggplot(subset(all_ages_df, age_group=="high" & sex=="b" & year >= 2019), aes(date, get_percentage(centered))) + geom_line(alpha=0.8) + 
# geom_smooth(method="lm", formula=y ~ ns(x, df=16), se=FALSE) +
geom_ribbon(aes(ymin=get_percentage(centered_low), ymax=get_percentage(centered_high)), alpha=0.3) + 
labs(x="Date", y="Deaths [% from baseline 2014-2019]", title="Anomalous Mortality 2012-2022") + 
facet_wrap(~country, scales="free_y")
p
ggsave(p,file="../figures/all_countries_smooth_high_age.pdf", width=25, height=10)

In [ ]:
options(repr.plot.width=25, repr.plot.height=10)
p = ggplot(subset(mid_ages_df, sex=="b" & year >= 2019), aes(date, get_percentage(centered))) + geom_line(alpha=0.8) + 
# geom_smooth(method="lm", formula=y ~ ns(x, df=16), se=FALSE) +
geom_ribbon(aes(ymin=get_percentage(centered_low), ymax=get_percentage(centered_high)), alpha=0.3) + 
labs(x="Date", y="Deaths [% from baseline 2014-2019]", title="Anomalous Mortality 2012-2022") + 
facet_wrap(~country, scales="free_y")
p
ggsave(p,file="../figures/all_countries_smooth_mid_age.pdf", width=25, height=10)

In [ ]:
# Cross-validate to find best smoothing parameter by minimizing out-of-sample 
# deviation from estimated exceedence. I.e. for a 95% confidence band, we will expect 
# 5% of out-of-sample estimates to be outside this band. We thus choose the smallest number 
# of parameters that get us there.
# Conclusion: The predictive power is not dependent on degrees of freedom of smoothing
# options(repr.plot.width=7, repr.plot.height=4)
cv_mortality = function(cdata,
                       years_per_df = 5,
                       test_start_year = 2020,
                       cv_type="week",  # week or "res" for residuals spline degrees of freedom
                       dfs=c(2, 4, 6, 8, 10, 12, 14, 16, 18, 20),
                       df_other=2,
                       response="Sdeaths_norm") {

    # TODO: make df_opther into a vector of same length as dfs
    
    all = cdata %>% rename(week=Week, year=Year, age=Age, date=Date, country=CountryCode, sex=Sex, 
                     response=all_of(response))
    all$week = as.numeric(all$week)
    all$year = as.numeric(all$year)
    
    df.yearly = floor(max((test_start_year - min(all$year)) / years_per_df, 2))
    train = subset(all, year < 2020 & year > 2012)
    alpha = 0.05 # Confidence interval

    years = unique(train$year)
    n_years = length(years)
    n_dfs = length(dfs)
    te_hat = matrix(0, n_years, n_dfs)
    tr_hat = matrix(0, n_years, n_dfs)
    sd_ratio_hat = matrix(0, n_years, n_dfs)
    INFL = 1.
    out = vector(mode="list", length=n_years)
    if (cv_type == "week") {
        dfs_week = dfs
        dfs_res = rep(df_other, length(dfs))
    } else if (cv_type == "res") {
        dfs_res = dfs
        dfs_week = rep(df_other, length(dfs))
    }

    for (i in 1:n_years) {
        out[[i]] = vector(mode="list", length=n_dfs)

        for (j in 1:n_dfs) {
            m_year = years[i]
            
            tr = subset(train, year != m_year)
            te = subset(train, year == m_year)
            n_tr = nrow(tr)
            n_te = nrow(te)
            yearly.fit = rq(response ~ ns(date, df=df.yearly), data=tr)

            tr[, "res.yearly"] = yearly.fit$res
            tr[, "fit.yearly"] = yearly.fit$fit

            if (dfs_week[j] <= 2) {
                weekly.fit = rq(res.yearly ~ week, data=tr)
            } else {
                weekly.fit = rq(res.yearly ~ 0 + pbs(week, df=dfs_week[j]), data=tr)
            }

            tr[, "fit.weekly"] = weekly.fit$fit
            tr[, "res.weekly"] = weekly.fit$res
            tr = tr %>% mutate(centered = response - fit.weekly - fit.yearly)
            te$fit.yearly = predict(yearly.fit, newdata=te)
            te$fit.weekly = predict(weekly.fit, newdata=te)

            te = te %>% mutate(centered = response - fit.weekly - fit.yearly)

            if (dfs_res[j] <= 2) {
                fit.low = quantile(tr$centered, alpha/2)
                fit.high = quantile(tr$centered, 1 - alpha/2)      
                tr$high = rep(fit.high, n_tr)
                tr$low = rep(fit.low, n_tr)
                te$high = rep(fit.high, n_te)
                te$low = rep(fit.low, n_te)
            } else {
                fit.low = rq(centered ~ 0 + pbs(week, df=dfs_res[j]), data=tr, tau=alpha/2)
                fit.high = rq(centered ~ 0 + pbs(week, df=dfs_res[j]), data=tr, tau=1-alpha/2)
                tr$high = fit.high$fit
                tr$low = fit.low$fit
                te$high = predict(fit.high, newdata=te)
                te$low = predict(fit.low, newdata=te)
            }
            
            sd_ratio_hat[i, j] = sd(te$centered) / sd(tr$centered)
            tr_hat[i, j] = 1 - (sum(tr$low > tr$centered | tr$high < tr$centered) / nrow(tr))
            te_hat[i, j] = 1 - (sum(te$low > te$centered | te$high < te$centered) / nrow(te))
        }
    }
    
    te_oos_error = data.frame(mean=apply(te_hat, 2, median), upper=apply(te_hat, 2, quantile, .75), lower=apply(te_hat, 2, quantile, .25), dfs=dfs)
    tr_oos_error = data.frame(mean=apply(tr_hat, 2, median), upper=apply(tr_hat, 2, quantile, .75), lower=apply(tr_hat, 2, quantile, .25), dfs=dfs)
    oos_error = rbind(train=tr_oos_error, test=te_oos_error)
    oos_error$data_set = rownames(oos_error)
    oos_error$data_set = gsub("\\.*\\d", "", oos_error$data_set)
    oos_error$sex = all$sex[1]  # all rows in input must be same sex
    oos_error$country = all$country[1]  # all rows in input must be same sex
    return(oos_error)
}

In [ ]:
oos_error = cv_mortality(high_ages %>% filter(CountryCode == "USA" & Sex == "b"))
tail(oos_error)

In [ ]:
oos_error = cv_mortality(high_ages %>% filter(CountryCode == "USA" & Sex == "b"), cv_type="res", df_other=4)
tail(oos_error)

In [ ]:
cv_high_ages = subset(high_ages, Sex=="b") %>% group_by(CountryCode) %>% group_map(~ cv_mortality(.x), keep=TRUE)

In [ ]:
# Cross-validate residual envelopes/quantiles degrees of freedom while keeping the weekly estimates fixed
cv_res_high_ages = subset(high_ages, Sex=="b") %>% group_by(CountryCode) %>% 
    group_map(~ cv_mortality(.x, cv_type="res", df_other=4), keep=TRUE)

In [ ]:
options(repr.plot.width=20, repr.plot.height=18)
p = ggplot(do.call(rbind, cv_high_ages),
      aes(x=dfs, mean, ymin=lower, ymax=upper, color=data_set)) + geom_errorbar() +
    theme_bw() +
    facet_wrap(~country) +
    labs(y="Probability of exceedence", x="Weekly spline degrees of freedom", 
    title="Estimation of model coverage")
p
ggsave(p, file="../figures/all_countries_cv.pdf", width=20, height=15)

In [ ]:
options(repr.plot.width=20, repr.plot.height=18)
p = ggplot(do.call(rbind, cv_res_high_ages),
      aes(x=dfs, mean, ymin=lower, ymax=upper, color=data_set)) + geom_errorbar() +
    theme_bw() +
    facet_wrap(~country) +
    labs(y="Probability of exceedence", x="Weekly spline degrees of freedom", 
    title="Estimation of model coverage")
p
ggsave(p, file="../figures/all_countries_cv_res.pdf", width=20, height=15)

In [ ]:
options(repr.plot.width=9, repr.plot.height=6)
ggplot(do.call(rbind, cv_all_ages) %>% filter(country=="USA"), 
       aes(x=dfs, mean, ymin=lower, ymax=upper, color=data_set)) + geom_errorbar() +
theme_bw() +
theme(plot.title=element_text(hjust=.5), 
      legend.key.size=unit(0.7, "cm"),
#      legend.position = c(0.18, 0.15),
     legend.title=element_blank()) + 
labs(y="Probability of exceedence", x="Weekly spline degrees of freedom", 
     title="Estimation of model coverage") +
scale_color_discrete("", labels=c("Test", "Train"))
# ggsave(get_figloc("cv_pois_s2008.pdf"), width=7, height=4)
# ggsave(get_figloc("cv_pois_s2008.png"), width=7, height=4)

In [ ]:
options(repr.plot.width=30, repr.plot.height=21)
p = ggplot(subset(high_ages_df, sex != "b" & year >= 2019), aes(date, get_percentage(centered)), color=sex) + geom_line(aes(color=sex)) + 
# geom_smooth(method="lm", formula=y ~ ns(x, df=16), se=FALSE) +
geom_ribbon(aes(ymin=get_percentage(centered_low), ymax=get_percentage(centered_high), color=sex), alpha=0, linetype=2) + 
labs(x="Date", y="Deaths [% from baseline 2014-2019]", title="Anomalous Mortality 2012-2022") + 
facet_wrap(~country, scales="free_y") + scale_color_discrete("", labels=c("Female", "Male"))
p
ggsave(p, file="../figures/all_countries_smooth_high_age_by_gender.pdf", width=25, height=10)

In [ ]:
options(repr.plot.width=30, repr.plot.height=21)
p = ggplot(subset(all_ages_df, sex == "b" & year >= 2019), aes(date, get_percentage(centered)), color=age_group) + geom_line(aes(color=age_group)) + 
# geom_smooth(method="lm", formula=y ~ ns(x, df=16), se=FALSE) +
geom_ribbon(aes(ymin=get_percentage(centered_low), ymax=get_percentage(centered_high), color=age_group), alpha=0, linetype=2) + 
labs(x="Date", y="Deaths [% from baseline 2014-2019]", title="Anomalous Mortality 2012-2022") + 
facet_wrap(~country, scales="free_y") + scale_color_discrete("", labels=c("High (65+)", "Mid (15-64)"))
p
ggsave(p, file="../figures/all_countries_smooth_both_gender_by_age_type.pdf", width=25, height=10)

In [ ]:
# Fit single country and sex
options(repr.plot.width=7, repr.plot.height=4)
cdata = subset(high_ages, CountryCode=="NOR" & Sex=="b")
all = mortality_fit(cdata)
plot_mortality(all)
plot_debug(all)

In [ ]:
# Write data for possible further analysis.
head(all_ages_df)
write.csv(all_ages_df, "../data/mortality_all_ages_normalized.csv")
write.csv(stratified, "../data/mortality_stratified_normalized.csv")